In [69]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector

# LLM model setting
chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks= [
    StreamingStdOutCallbackHandler()
  ],
)

# give the example to FewShotChat. we can bring many other example from some data base
examples = [
  {
    "question": "France",
    "answer" : """
    Here is what i know :
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency : Euro
    """,
  },
  {
    "question":"Italy",
    "answer": """
    Here is what i know:
    Capital : Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency : Euro
    """,
  },
  {
    "question":"Korea",
    "answer" : """
    Here is what i know:
    Capital : Seoul
    Language : Korean
    Food: Kimchi and Samgyopsal
    Currency : Won
    """
  }
]

class RandomExampleSelector(BaseExampleSelector):

  def __init__(self, examples):
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice
    return [choice(self.examples)]

  
example_prompt = PromptTemplate.from_template("human: {question} \n AI: {answer}")

example_selector = RandomExampleSelector(
  examples=examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"]
)

# chain = prompt | chat

prompt.format(country = "Russia")

# chain.invoke({
#   "country" : "Russia"
# })


'human: Italy \n AI: \n    Here is what i know:\n    Capital : Rome\n    Language: Italian\n    Food: Pizza and Pasta\n    Currency : Euro\n    \n\nHuman: What do you know about Russia?'